In [1]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.regularizers import l2
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import keras_tuner as kt

# set random seed for reproducibility
np.random.seed(69)
tf.random.set_seed(69)


In [7]:
## Data Load and Preprocess
# function to load and preprocess frame level data (lets get freaky)
def load_frame_data(folders):
    """ 
    loads csvs as individual frames, removes frames where more than 75% of columns are zero 
    """
    data, labels = [], []

    for folder, label in folders:
        if os.path.exists(folder):
            for file in os.listdir(folder):
                if file.endswith('.csv'):
                    file_path = os.path.join(folder, file)
                    df = pd.read_csv(file_path)

                    # remove 'frame' column and keep only movement data
                    features = df.iloc[:, 2:].values  

                    # remove frames where more than 75% of columns are zero
                    zero_threshold = 0.75  # 75% threshold
                    valid_rows = np.mean(features == 0, axis=1) < zero_threshold
                    features = features[valid_rows]

                    if len(features) > 0:
                        data.extend(features)
                        labels.extend([label] * len(features))
        else:
            print(f"warning: folder {folder} not found.")

    return np.array(data), np.array(labels)

# define training folders
train_folders = [
    ('../rat_dance_csv/train', 1),
    ('../neg_control_csv/train', 0)
]

# load dataset
X, y = load_frame_data(train_folders)

# normalize features
scaler = StandardScaler()
X = scaler.fit_transform(X)  # normalize across all frames

# shuffle dataset
indices = np.random.permutation(len(X))
X, y = X[indices], y[indices]

# store feature count for model input shape
num_features = X.shape[1]


In [8]:
## Define model
# function to build a fully connected neural network
def build_model(hp):
    model = keras.Sequential([
        # first dense layer
        keras.layers.Dense(
            units=hp.Int("units_1", min_value=32, max_value=128, step=16), 
            activation="relu", kernel_regularizer=l2(0.01), input_shape=(num_features,)
        ),
        keras.layers.BatchNormalization(),
        keras.layers.Dropout(hp.Float("dropout_1", 0.3, 0.6, step=0.1)),

        # second dense layer
        keras.layers.Dense(
            units=hp.Int("units_2", min_value=16, max_value=64, step=16), 
            activation="relu", kernel_regularizer=l2(0.01)
        ),
        keras.layers.BatchNormalization(),
        keras.layers.Dropout(hp.Float("dropout_2", 0.3, 0.6, step=0.1)),

        # output layer
        keras.layers.Dense(1, activation="sigmoid"),
    ])

    # compile model
    model.compile(
        optimizer=keras.optimizers.Adam(
            learning_rate=hp.Choice("learning_rate", [0.001, 0.0005, 0.0001])
        ),
        loss="binary_crossentropy",
        metrics=["accuracy"],
    )

    return model

In [10]:
## Tuning Hyperparams
# use keras tuner to search for best hyperparameters
tuner = kt.tuners.RandomSearch(
    hypermodel=build_model,
    objective="val_loss", 
    max_trials=10, 
    executions_per_trial=2,
    directory="delete_me_post_search_three",
    project_name="dense_nn_tuning",
)

# split data for tuning
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=69)
train_index, val_index = next(kf.split(X, y)) 
X_train, X_val = X[train_index], X[val_index]
y_train, y_val = y[train_index], y[val_index]

# search for best hyperparameters
tuner.search(X_train, y_train, epochs=10, batch_size=128, validation_data=(X_val, y_val), verbose=1)

# get best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# print best values
print(f"Best units (layer 1): {best_hps.get('units_1')}")
print(f"Best dropout (layer 1): {best_hps.get('dropout_1')}")
print(f"Best units (layer 2): {best_hps.get('units_2')}")
print(f"Best dropout (layer 2): {best_hps.get('dropout_2')}")
print(f"Best learning rate: {best_hps.get('learning_rate')}")


Trial 10 Complete [00h 00m 06s]
val_loss: 0.18949826806783676

Best val_loss So Far: 0.17101025581359863
Total elapsed time: 00h 00m 54s
Best units (layer 1): 96
Best dropout (layer 1): 0.4
Best units (layer 2): 16
Best dropout (layer 2): 0.3
Best learning rate: 0.001


In [12]:
# Create final model
# create final model using best hyperparameters
final_model = tuner.hypermodel.build(best_hps)

# define early stopping
early_stopping = keras.callbacks.EarlyStopping(
    monitor="val_loss", patience=5, restore_best_weights=True
)

# define reduce learning rate callback
reduce_lr = keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss", factor=0.5, patience=3, min_lr=1e-6
)

# train final model on full dataset
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=69)
cv_accuracies = []

for train_index, val_index in kf.split(X, y):
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]

    final_model.fit(X_train, y_train, epochs=15, batch_size=128, verbose=0, validation_data=(X_val, y_val), callbacks=[early_stopping, reduce_lr])

    # evaluate the model
    val_loss, val_acc = final_model.evaluate(X_val, y_val, verbose=0)
    cv_accuracies.append(val_acc)

# print cross validation accuracy
cross_val_acc = np.mean(cv_accuracies)
print(f"final cross validation accuracy: {cross_val_acc:.4f}")


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


final cross validation accuracy: 0.9872


In [13]:
## validate model on validation set
# load validation dataset
val_folders = [
    ('../rat_dance_csv/val', 1),
    ('../neg_control_csv/val', 0)
]

X_val, y_val = load_frame_data(val_folders)
X_val = scaler.transform(X_val)  

# evaluate final model on validation set
y_pred_prob = final_model.predict(X_val)
y_pred = (y_pred_prob > 0.5).astype(int)  

accuracy = accuracy_score(y_val, y_pred)
class_report = classification_report(y_val, y_pred, target_names=["negative control (0)", "ratdance (1)"])
conf_matrix = confusion_matrix(y_val, y_pred)

# show results
print(f"\nValidation Performance:")
print(f"Accuracy: {accuracy:.4f}")
print("\nClassification Report:")
print(class_report)
print("\nConfusion Matrix:")
print(conf_matrix)

155/155 ━━━━━━━━━━━━━━━━━━━━ 0s 544us/step

Validation Performance:
Accuracy: 0.7765

Classification Report:
                      precision    recall  f1-score   support

negative control (0)       0.76      0.80      0.78      2472
        ratdance (1)       0.79      0.75      0.77      2472

            accuracy                           0.78      4944
           macro avg       0.78      0.78      0.78      4944
        weighted avg       0.78      0.78      0.78      4944


Confusion Matrix:
[[1982  490]
 [ 615 1857]]


In [14]:
## Validate model on test set
# load test dataset
test_folders = [
    ('../rat_dance_csv/test', 1),
    ('../neg_control_csv/test', 0)
]

X_test, y_test = load_frame_data(test_folders)
X_test = scaler.transform(X_test)  

# evaluate final model on test set
y_test_pred_prob = final_model.predict(X_test)
y_test_pred = (y_test_pred_prob > 0.5).astype(int)  

test_accuracy = accuracy_score(y_test, y_test_pred)
test_class_report = classification_report(y_test, y_test_pred, target_names=["negative control (0)", "ratdance (1)"])
test_conf_matrix = confusion_matrix(y_test, y_test_pred)

# display results
print(f"\nTest Set Performance:")
print(f"Accuracy: {test_accuracy:.4f}")
print("\nClassification Report:")
print(test_class_report)
print("\nConfusion Matrix:")
print(test_conf_matrix)


153/153 ━━━━━━━━━━━━━━━━━━━━ 0s 328us/step

Test Set Performance:
Accuracy: 0.7938

Classification Report:
                      precision    recall  f1-score   support

negative control (0)       0.86      0.70      0.77      2403
        ratdance (1)       0.75      0.89      0.81      2485

            accuracy                           0.79      4888
           macro avg       0.80      0.79      0.79      4888
        weighted avg       0.80      0.79      0.79      4888


Confusion Matrix:
[[1671  732]
 [ 276 2209]]
